# TRABALHO #2 — Grupo 6

 **Tópico: Decision Trees**

Membros:

 - Mariana Filipa Ribeiro Ferreira (up202205006)
 - Raquel Barbosa Ferreira Alves   (up202206827)
 - Gonçalo Reimão Silva da Luz     (up202205522)

 Vídeo:
 - https://drive.google.com/file/d/1VsTrIsfLVJjSS7ZkxIvVFV_YH-9eV3Pb/view?usp=sharing

In [ ]:
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from math import log2

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

rows_destruidas = 0

def discValues(data):
    num_bins = int(np.log2(len(data)) + 1)
    bin_intervals = {}
    
    for column in data.columns[1:]:
        if pd.api.types.is_numeric_dtype(data[column]):  # Apenas discretiza colunas numéricas
            # Discretiza os valores e obtém os bins
            discretized_values, bins = pd.cut(data[column], bins=num_bins, labels=False, retbins=True)
            data[column] = discretized_values
            bin_intervals[column] = bins
    
    return data, bin_intervals

dataset_paths = {
    'restaurant': r"restaurant.csv",
    'weather': r"weather.csv",
    'iris': r"iris.csv",
    'connect4': r"connect4_2.csv", # connect4 mas com a primeira linha
    'new': r"new.csv"              # alterar new para nome do ficheiro
}

while True:
    choice = input("Choose the dataset: ")

    if choice in dataset_paths:
        if choice == 'restaurant': # trocar os none para NONE
            csv_data = pd.read_csv('restaurant.csv', dtype={'Pat': str}, na_filter=False)
            csv_data['Pat'] = csv_data['Pat'].apply(lambda x: 'NONE' if x == 'None' else x)
            csv_data.to_csv('restaurant.csv', index=False)
        else:
            csv_data = pd.read_csv(dataset_paths[choice])

        headers = csv_data.columns.tolist()
        dataset = csv_data.values.tolist()

        num_items = len(dataset[0])
        cleaned_dataset = []

        csv_data, bin_intervals = discValues(csv_data)
        
        for row in dataset:
            if len(row) == num_items:
                cleaned_dataset.append(row)
            else:
                rows_destruidas += 1

        print("Dataset loaded, cleaned, and discretized successfully.")
        print(f"Number of disregarded rows: {rows_destruidas}")
        print("- - - - - -")
        for column, bins in bin_intervals.items():
            print(f"{column}: {bins}")
        print("- - - - - -")
        break
    else:
        print("Invalid choice. Please choose a valid dataset.")

Três bibliotecas são importadas no início do ficheiro:
 - A biblioteca "pandas" é usada para manipulação e analise de dados.
 - A biblioteca "numpy" é usada para operações numéricas (em específico, na função "discValues").
 - Da biblioteca "collections", são importados "Counter" e "collections", que são utilizados para a contagem de elementos e a criação de dicionários.
 - Da biblioteca "math" é importado "log2", necessário para calcular a entropia.

A variável "choice" requer input do utilizador, de um dos dataset especifícados no dicionário "dataset_paths". Estes dicionário mapeia uma palavra para o ficheiro associado (exemplo: a chave "restaurant" indica o ficheiro "restaurant.csv" que contém o dataset associado).

Se a escolha guardada em "choice" for inválida, é impressa uma mensagem de erro. Se for válida, o programa prossegue e ficheiro escolhido é lido pela função "pd.read_csv()", que cria a DataFrame "csv_data" com a informação.

A variável "headers" recebe um array com os nomes dos atríbutos do DataFrame criado, enquanto a variável "dataset" converte o DataFrame num array de arrays, onde cada sub-array representa uma linha do dataset original.

De seguida, o dataset é analisado e, através da função "discValues". Esta função discretiza os valores numéricos usando a fórmula de Sturges. De seguida, se alguma fila apresentar falhas, esta é apagada, o que será contado na variável "rows_destruidas".

De seguida, é impresso o nnúmero de linhas eliminadas, e os bins criados pela função "discValues".

## Entropy

In [ ]:
def entropy(data):
    total = len(data)
    counts = Counter(row[-1] for row in data)
    return -sum((count/total) * log2(count/total) for count in counts.values())

A função "entropy" recebe o dataset e calcula a sua entropia associada.

A variável "total" contém o número de linhas do dataset recebido. A variável "count" conta cada occorência de cada classe na última coluna. Com isso, a função retorna o valor da entropia através da fórmula:
 - -sum((count/total) * log2(count/total) for count in counts.values())

## Information Gain

In [ ]:
def information_gain(data, attribute_index):
    total = len(data)
    subsets = defaultdict(list)
    
    for row in data:
        subsets[row[attribute_index]].append(row)
    
    weighted_entropy = sum((len(subset)/total) * entropy(subset) for subset in subsets.values())
    return entropy(data) - weighted_entropy

A função "information_gain" recebe o dataset e o index do atríbuto cujo ganho de informação queremos determinar.

A variável "total" contém o número de linhas do dataset recebido. A variável "subsets" contém um dicionário de um subconjunto do dataset, dividido pelos valores únicos do atributo especificado (gerado no loop seguinte).

De seguida, é calculada a entropia de cada subconjunto e, ao subtrair essa à entropia do dataset todo, temos o ganho de informação do atributo especfico.

## Best Attribute

In [ ]:
def best_attribute(data, attributes):
    gains = [(attr, information_gain(data, index)) for index, attr in enumerate(attributes)]
    return max(gains, key=lambda x: x[1])

A função "best_attribute" recebe o dataset e todos os seus atributos, calculando (com chamadas à funçao anterior) qual dos atributos do dataset tem o maior ganho de informação.

In [ ]:
print("Entropy of the dataset:", entropy(dataset))
print("Best attribute to split on:", best_attribute(dataset, headers[1:-1]))
print("- - - - - -")

## Class "Node"

In [ ]:
class Node:
    def __init__(self, attribute=None, value=None, result=None, counter=None, branches=None):
        self.attribute = attribute
        self.value = value
        self.result = result
        self.counter = counter
        self.branches = branches if branches is not None else {}

É definida uma estrutura "Node" que representa um nó na árvore de decisão. Cada nó tem associado um atributo "attributte", valor "value", resultado "result, contador "counter", e ramos "branches (que se não for inicializado fica um dicionário vazio).

## Build Tree

In [ ]:
def build_tree(data, attributes):
    if not data:
        return None
    
    class_counts = Counter(row[-1] for row in data)
    if len(class_counts) == 1:
        return Node(result=list(class_counts.keys())[0], counter=len(data))
    if not attributes:
        result = class_counts.most_common(1)[0][0]
        return Node(result=result, counter=len(data))
    best_attr, best_gain = best_attribute(data, attributes)
    best_attr_index = headers.index(best_attr)
    root = Node(attribute=best_attr)
    partitions = defaultdict(list)
    for row in data:
        partitions[row[best_attr_index]].append(row)
    
    for attr_value, subset in partitions.items():
        remaining_attrs = [attr for attr in attributes if attr != best_attr]
        # int --> string
        if isinstance(attr_value, int):
            attr_value = str(attr_value)
        branch = build_tree(subset, remaining_attrs)
        root.branches[str(attr_value)] = branch  # --> string
    return root


A função "build_tree" recebe o dataset e os seus atributos. Se o dataset for vazio, retorna "None". Este é o caso base da função "build_tree", que constroi a árvore de decisão de modo recursivo.

Se o dataset não for vazio, a variável "class_counts" conta o número de ocorrências de cada classe no dataset. Se todas as linhas do dataset pertencerem à mesma class, um nó filho é criado com essa classe como "result" e o número de filas como "counter".

Se não existirem mais atributos, um nó filho é criado com a classe mais comum como "result" e o número de filas como "counter".

De seguida, é encontrado o melhor atributo para dividir a árvore através da função "best_attribute" e o seu index associado. Esse atributo será a raiz da árvore de decisão criada. Um dicionário é inicializado para guardar as partições do dataset e, para cada partição, a função constroi recursivamente a árvore de decisão com os métodos acima mencionados.

A variável "attributes" cria a lista que será utilizada para construir a árvore de decisão (que contém o "header" do ficheiro .csv dado, excluindo a primera e última coluna). Esta função retorna a raiz da árvore. 

## Print Tree

In [ ]:
attributes = headers[1:-1]
decision_tree = build_tree(dataset, attributes)

def print_tree(node, indent="   "):
    if node.result is not None:
        print(f"{indent}{node.result}: (Counter: {node.counter})")
    else:
        print(f"{indent}<{node.attribute}>")
        for value, branch in node.branches.items():
            print(f"{indent}{value}:")
            print_tree(branch, indent + "    ")

print_tree(decision_tree)

A variável "attributes" excluí a primeira e última colunas de "headers". Com esta variável e a variável "dataset", usamos a função "build_tree" para construir a árvore de decisão correspondente, que fica disponível a partir da variável "decision_tree".

A função "print_tree" imprime recursivamente a árvore de decisão. Para folhas, imprime o resultado e o counter associado. Para nós de decisão, imprime o atributo associado e, recursivamente, cada ramo.

## Predict Single

In [ ]:
def PredictSingle(instance, decision_tree):
    if decision_tree.result is not None:
        return decision_tree.result
    
    attribute = decision_tree.attribute
    attr_value = instance.get(attribute)

    if (attr_value is None) or (attr_value not in list(decision_tree.branches.keys())):
        next
    else:
        return PredictSingle(instance, decision_tree.branches[attr_value])

A função "PredictSingle" é uma função recursiva que recebe uma potencial instancia de um cenário, e a árvore de decisão associada com esse mesmo ficheiro.

A função começa por verificar se o nó atual da árvore de decisão contém um "result". Se sim, esse resultado é retornado diretamente. Se não, então a função atravessa a árvore recursivamente, passando para os nós filhos.

## Predict File

In [ ]:
def predict_file(input_file, decision_tree):
    predictions = []
    
    with open(input_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        instance = {}
        attributes = line.strip().split(',')
        for index, attr_value in enumerate(attributes):
            instance[headers[index+1]] = attr_value
        
        predicted_class = PredictSingle(instance, decision_tree)
        predictions.append(predicted_class)
    
    return predictions

# '''
print("- - - - - -")



A função "predict_file" lê um ficheiro que contém várias instâncias de um cenário e, utilizando a função "PredictSingle", verifica cada instância e retorna uma lista com as classes previstas para cada uma dessas instancias.

## Prediction Tests

In [ ]:
if choice == 'restaurant':
    predictions = predict_file(r"test1.csv", decision_tree)
    print(f"Predicted classes: ", predictions)

elif choice == 'weather':
    predictions = predict_file(r"test2.csv", decision_tree)
    print(f"Predicted classes: ", predictions)

elif choice == 'iris':
    predictions = predict_file(r"test3.csv", decision_tree)
    print(f"Predicted classes: ", predictions)

elif choice == "connect4":
    instance = {
    'X1': 'b',
    'X2': 'b',
    'X3': 'b',
    'X4': 'b',
    'X5': 'b',
    'X6': 'b',
    'X7': 'b',
    'X8': 'b',
    'X9': 'b',                #b,b,b,b,b,b,b,b,b,b,b,b,x,o,b,b,b,b,x,o,x,o,x,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,win
    'X10': 'b',
    'X11': 'b',
    'X12': 'b',
    'X13': 'x',
    'X14': 'o',
    'X15': 'b',
    'X16': 'b',
    'X17': 'b',
    'X18': 'b',
    'X19': 'x',
    'X20': 'o',
    'X21': 'x',
    'X22': 'o',
    'X23': 'x',
    'X24': 'o',
    'X25': 'b',
    'X26': 'b',
    'X27': 'b',
    'X28': 'b',
    'X29': 'b',
    'X30': 'b',
    'X31': 'b',
    'X32': 'b',
    'X33': 'b',
    'X34': 'b',
    'X35': 'b',
    'X36': 'b',
    'X37': 'b',
    'X38': 'b',
    'X39': 'b',
    'X40': 'b',
    'X41': 'b',
    'X42': 'b'
    }
    predictions = PredictSingle(instance, decision_tree)
    print(f"Predicted classes: ", predictions)

elif choice == 'new':
    predictions = predict_file(r"test_new.csv", decision_tree)   #colocar teste para o ficheiro new
    print(f"Predicted classes: ", predictions)

## Evaluate Model

In [ ]:
def prepare_data(df):
    data_with_labels = df.values.tolist()
    train_data, test_data = train_test_split(data_with_labels, test_size=0.3)
    X_test = [row[1:-1] for row in test_data]
    y_test = [row[-1] for row in test_data]
    y_test = [str(label) for label in y_test]
    return train_data, X_test, y_test

A função "prepare_data" é utilizada para preparar os dados para o treino e teste da árvore criada. A função retorna train_data (os dados de treino), X_test (os atributos usados para o teste) e y_test (as classes do conjunto de teste).

In [ ]:
def make_predictions(instances, decision_tree):
    predictions = []
    for instance in instances:
        prediction = PredictSingle(instance, decision_tree)
        predictions.append(prediction)
    return predictions

Esta função faz previsões para uma lista de instâncias usando a árvore de decisão. O seu objetivo é prever classes para um conjunto de dados de teste.

In [ ]:
def evaluate_model(df, test_size):
    train_data, X_test, y_test = prepare_data(df)
    attributes = headers[1:-1]
    train_tree = build_tree(train_data, attributes)

    instances = [{attribute: value for attribute, value in zip(attributes, instance)} for instance in X_test]

    predictions = make_predictions(instances, train_tree)
    predictions = [str(pred) for pred in predictions]

    accuracy = sum([1 for i in range(len(y_test)) if y_test[i] == predictions[i]]) / len(y_test)
    print(f"Accuracy: {accuracy:.2f}")

    balanced_accuracy = balanced_accuracy_score(y_test, predictions)
    print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

    print(str(predictions) + "\n")
    print(str(y_test) + "\n")
    predictions_correct = []
    for true_label, predicted_label in zip(y_test, predictions):
        if true_label == predicted_label:
            predictions_correct.append(1)
        else:
            predictions_correct.append(0)
    predictions_total = len(predictions_correct)
    correct_count = sum(predictions_correct)
    incorrect_count = predictions_total - correct_count

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='macro', zero_division=0)
    recall = recall_score(y_test, predictions, average='macro', zero_division=0)
    f1 = f1_score(y_test, predictions, average='macro', zero_division=0)
    
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    # gráfico de barras
    plt.bar(['Corretas', 'Incorretas'], [correct_count, incorrect_count], color=['green', 'red'])
    plt.title('Previsões Corretas vs. Previsões Incorretas')
    plt.xlabel('Tipo de Previsão')
    plt.ylabel('Número de Previsões')
    plt.show()

print("- - - - - -")
evaluate_model(csv_data, test_size=0.3)

A função "evaluate_model" avalia o desempenho de uma árvore de decisão usando um conjunto de dados fornecido. Os argumentos passados à função são o dataframe com as informações necessárias e o tamanho desejado para o conjunto de teste.
Começa por preparar os dados para que os possa usar para fazer previsões. Depois de feitas, calcula a precisão com o qual a árvore acertou na classe, comparando predictions com y_test. 
Por fim, constrói um gráfico de barras que ilustra o número de previsões corretas e incorretas.
